# Session 1: MPS class

Some requirements for this library component. We use the `numpy` library and import algorithms to operate on matrix product states, such as scalar produts, expectation values, norms, etc.

In [1]:
# file: mps/state.py
import numpy as np

## General idea

A Matrix-Product State (abbreviated MPS), is a representation of quantum states as a one-dimensional contraction of tensors. Imagine the following quantum state
<img src="figures/fig-state.png" style="max-width: 50%; width:400px">

The state has four physical indices, so that we can write it as
$$|\Psi\rangle = \sum_{i_1,i_2,i_3,i_4} \Psi_{i_1,i_2,i_3,i_4} |i_1,i_2,i_3,i_4\rangle.$$
If the physical dimension of each component is $d$, that is $i_k \in \{0,1,\ldots, d-1\}$, this means that the total state requires $d^4$ complex numbers to be represented.


The MPS representation uses a small number of tensors (matrices) to reconstruct the previous state as
<img src="figures/fig-mps.png" style="max-width: 50%; width: 400px">

Now the state reads
$$|\Psi\rangle = \sum_{i's, \alpha's} A^{i_1}_{\alpha_1} B^{i_2}_{\alpha_1,\alpha_2} C^{i_4}_{\alpha_2,\alpha_3} D^{i_3}_{\alpha_3}|i_1,i_2,i_3,i_4\rangle.$$
This can be simply written in a more compact fashion as
$$
|\Psi\rangle = A^{i_1} B^{i_2} C^{i_3} D^{i_4}|i_1,i_2,i_3,i_4\rangle,
$$
assuming that the $\{A,B,C,D\}$ are either vectors or matrices, labeled by the physical index $i_k$, and that we contract over repeated indices.

## Tensor arrays

The first step before creating more complicated structures is to ensure that we have the right logic when studying these tensor arrays. Our contract is as follows:

* The tensor network can be destructively modified. That is, we can replace tensors in the network after it has been created.

* The tensor network can be cloned, creating a fresh new copy that shares the same tensor.

* A clone of a tensor network only shares the tensors with its sibling. We can modify the sibling and the clone replacing tensors, without one affecting the other.

* We do not contemplate in-place modifications to the tensors themselves.

The class that implements this contract is shown below. It uses `__getitem__` and `__setitem__` to allow access to the class as if it was an array. It implements `__copy__()` to create fresh new copies, either with the library function `copy.copy()` or by directly using the method `clone()`.

In [2]:
# file: mps/state.py


class TensorArray(object):
    """TensorArray class.

    This class provides the basis for all tensor networks. The class abstracts
    a one-dimensional array of tensors that is freshly copied whenever the
    object is cloned. Two TensorArray's can share the same tensors and be
    destructively modified.

    Attributes:
    size = number of tensors in the array
    """

    def __init__(self, data):
        """Create a new TensorArray from a list of tensors. The list is cloned
        before storing it into this object, so as to avoid side effects when
        destructively modifying the array."""
        if type(data) is list:
            self._data = data.copy()
            self.size = len(self._data)
        else:
            raise TypeError()

    def __getitem__(self, k):
        #
        # Get MP matrix at position `k`. If 'A' is an MP, we can now
        # do A[k]
        #
        return self._data[k]

    def __setitem__(self, k, value):
        #
        # Replace matrix at position `k` with new tensor `value`. If 'A'
        # is an MP, we can now do A[k] = value
        #
        self._data[k] = value
        return value

    def __copy__(self):
        #
        # Return a copy of the MPS with a fresh new array.
        #
        return type(self)(self._data)

    def clone(self):
        """Return a fresh new TensorArray that shares the same tensor as its
        sibling, but which can be destructively modified without affecting it.
        """
        return self.__copy__()

## Simple MPS

An MPS is a tensor array that stores tensors with three indices. We will follow these conventions:

1. All tensors in MPS.data will have three indices, `A[α, i, β]`:
  - `(α, β)` are the virtual dimensions of the MPS
  - `i` is the physical dimension of the given site.
  
2. In general, we do not yet assume any normal form for the states.

3. We restrict ourselves to open-boundary-condition states, which have the following properties:
  - For the first site, `α=1`. For the last site `β=1`.
  - For the first site, α takes one value only, i.e. `α=0` in Python. For the last site `β=0` similarly.

All our implementations of MPS share the interface below, which provides methods for additional algorithms --scalar products, norms, expected values, etc-- that are developed in other components of the library.

In [3]:
# file: mps/state.py


class MPS(TensorArray):
    """MPS (Matrix Product State) class.

    This implements a bare-bones Matrix Product State object with open
    boundary conditions. The tensors have three indices, A[α,i,β], where
    'α,β' are the internal labels and 'i' is the physical state of the given
    site.

    Attributes:
    size = number of tensors in the array
    """

    #
    # This class contains all the matrices and vectors that form
    # a Matrix-Product State.
    #
    def __init__(self, data):
        super(MPS, self).__init__(data)

    def dimension(self):
        """Return the total size of the Hilbert space in which this MPS lives."""
        return np.product([a.shape[1] for a in self.data])

    def tovector(self):
        """Return one-dimensional complex vector of dimension() elements, with
        the complete wavefunction that is encoded in the MPS."""
        return _mps2vector(self)

    def norm2(self):
        """Return the square of the norm-2 of this state, ‖ψ‖**2 = <ψ|ψ>."""
        return mps.expectation.scprod(self.data, self.data)

    def expectation1(self, operator, n):
        """Return the expectation value of 'operator' acting on the 'n'-th
        site of the MPS."""
        return mps.expectation.expectation1_non_canonical(self, operator, n)

    def all_expectation1(self, operator):
        """Return all expectation values of 'operator' acting on all possible
        sites of the MPS."""
        return mps.expectation.all_expectation1_non_canonical(self, operator)

### Convert an MPS into a vector

This is our first algorithm. We write a function that convers an MPS into a complex vector $\Psi$ with all the components in it. The algorithm implements the full contraction of the tensors, from left to right (i.e. from position 0 to position L-1, where L is the `mps.size`).

For instance, if the MPS has two sites of dimension 2, the MPS will have two tensors $A_{0i\alpha}$ and $B_{\alpha{j}0}$ that are contracted together to give the state
$$|\psi\rangle = \sum_{i,j,\alpha,\beta} A_{0i\alpha}B_{\alpha{j}0}|i\rangle\otimes|j\rangle.$$

In [4]:
# file: mps/state.py


def _mps2vector(data):
    #
    # Input:
    #  - data: list of tensors for the MPS (unchecked)
    # Output:
    #  - Ψ: Vector of complex numbers with all the wavefunction amplitudes
    #
    # We keep Ψ[D,β], a tensor with all matrices contracted so far, where
    # 'D' is the dimension of the physical subsystems up to this point and
    # 'β' is the last uncontracted internal index.
    #
    Ψ = np.ones((1, 1,))
    D = 1
    for (i, A) in enumerate(data):
        α, d, β = A.shape
        Ψ = np.einsum('Da,akb->Dkb', Ψ, A)
        D = D * d
        Ψ = np.reshape(Ψ, (D, β))
    return Ψ.reshape((Ψ.size,))

Given our conventions, the wavefunction will be ordered as follows
$$\Psi = \left(\begin{array}{c}
\sum_\alpha A_{00\alpha} B_{\alpha00} \\
\sum_\alpha A_{00\alpha} B_{\alpha10} \\
\sum_\alpha A_{01\alpha} B_{\alpha00} \\
\sum_\alpha A_{01\alpha} B_{\alpha10}
\end{array}\right).$$

In [5]:
_mps2vector([np.reshape([1,2],(1,2,1)), np.reshape([3,5],(1,2,1))])

array([ 3.,  5.,  6., 10.])

This is the same convention as used by `np.kron()`

In [6]:
np.kron([1,2],[3,5])

array([ 3,  5,  6, 10])